<a href="https://colab.research.google.com/github/ggharibian/Graphing-Calculator-C/blob/main/Linda_Bessin_KNN_Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
*90 lines*

In [ ]:
# Run this cell to set up the notebook
import numpy as np
import pandas as pd
import math
from datascience import *
from google.colab import drive
drive.mount('/content/drive')
drive.mount('/content/drive', timeout_ms=9999999999999999999999999999999999999999999999999999999999999999999999999999)

# These lines set up the plotting functionality and formatting.
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

#Bessin-specific functions
def table_setup(t_name,):
    def tt2(s):
        return s[:-2]
        
    t = pd.read_csv('/content/drive/My Drive/Notebooks/Bessin/Data/' + t_name + '.csv')
    if 'PrecinctName' in t.columns:
        t['PrecinctName'] = t['PrecinctName'].apply(tt2)
    elif 'PrecinctName ' in t.columns:
        t['PrecinctName '] = t['PrecinctName '].apply(tt2)
    return t.rename({'Voter File VANID': 'VANID', 'FirstName': 'firstname', 'MiddleName': 'middlename', 'LastName': 'lastname', 'Suffix': 'suffix', 'Sex': 'sex', 'Age': 'age', 'Age ': 'age', 'mAddress': 'mailingaddress', 'mCity': 'city', 'mState': 'state', 'mZip5': 'zip', 'mZip4': 'zip_ext', 'PreferredEmail': 'preferredemail', 'Preferred Phone': 'preferred Phone', 'PrecinctName': 'precinct', 'RaceName': 'race', 'Party': 'party', 'Party ': 'party', '2016:CmpgnFnc': 'campaignfinancing', '2016:ClgFndng': 'collegefunding', '2016:GvPrvcy': 'govtprivacy', '2016:PdLv': 'paidleave', '2016:NonChrst': 'nochristian', '2016:OthChrst': 'otherchristian', '2016:TSM:TeaPa': 'teaparty', 'Autho': 'authoritarian', 'Comp': 'compassion', 'EcoPop': 'econpopulism', 'Env': 'environment', 'FrTrd': 'freetrade', 'Glbl': 'globalism', 'Guns': 'guns', 'HC': 'healthcare', 'HCWmn': 'healthcarewomen', 'Imm': 'immigration', 'Mili': 'military', 'Poor': 'poor', 'Popu': 'populism', 'Pres': 'presidential', 'RacRes': 'racialresentment', 'Reg': 'regulation', 'RelFrdm': 'religiousfreedom', 'Tax': 'tax', 'Trad': 'traditionalism', 'InstTrst': 'institutiontrust', 'ClmChng': 'climatechange', 'CollGrd': 'collegegrad', 'HSOnly': 'hsonly', 'Ideo': 'ideology', 'LocVo': 'localvoter', 'PthCit': 'pathtocitizen', 'PrChc': 'prochoice', 'ProgTax': 'progressivetax', 'WorCl': 'workingclass', 'Cosmo': 'cosmopolitanism', 'CrimJustRef': 'criminaljusticereform', 'Diplomacy': 'diplomacy', 'EthNat': 'ethnonationalism', 'IncInequ': 'incomeinequality', 'Infrastruct': 'infrastructure', 'TSMAct': 'activist', 'CanLeg': 'cannabislegalization', '2016:ChldPrsnt': 'childrenpresent', 'TSMGnCntrl': 'guncontrol', 'TSMGun': 'gunowner', 'TSM_Ideo_Enh': 'ideologyenhanced', '2016:IncRnk': 'incomerank', 'UnionSupport': 'unionsupport', '2016:Mrrg': 'marriage', 'MarEqu': 'marriageequality', 'MidGenTO': 'midtermturnout', 'MidGenTOEnth': 'midtermenthusiasm', '2016:MnWg': 'minimumwage', 'NonPresPriTO': 'nonprespriturnout', 'OffGenTO': 'offgeneralturnout', 'TSM:Part': 'partisan', 'PresGenTO': 'presgeneralturnout', 'PresPriTO': 'prespriturnout', '2016:Cath': 'catholic', '2016:Evang': 'evangelical', 'Jewish': 'jewish', 'Mormon': 'mormon', 'NonRelig': 'nonreligious', 'TSMTrumpResist': 'trumpresistance', 'TSMTrumpSup': 'trumpsupport', 'TSMVet': 'veteran', 'Urb ': 'urbanicity'}, axis='columns')

def precinct_breakdown(t, bkdn, pnt=None, in_percent=False):
    p = pd.DataFrame()
    if 'precinct' in t.columns:
        t = t[t['precinct'] == pnt]
        p['precinct'] = [pnt]
    for k in list(bkdn):
        if k in t.columns:
            for d in bkdn[k]:
                if d in list(t[k].unique()):
                    ct = t.groupby(k)['LastName'].count()[d]
                else:
                    ct = 0
                if in_percent == True and type(pnt) == str:
                    ct = ct / len(t[t['precinct'] == pnt].index)
                elif in_percent == True and not type(pnt) == str:
                    ct = ct / len(t.index)
                p[d] = [ct]
    return p

def mult_breakdown(tbl, bkdn, pr_list=[], in_percent=False):
    if pr_list == []:
        pr_list = list(tbl['precinct'].unique())
    df1 = pd.DataFrame()
    for i in pr_list:
        df1 = df1.append(precinct_breakdown(tbl, bkdn, i, in_percent), ignore_index=True)
    return df1

def zip_p1_full(t, dict): # NEEDS TO BE TRANSFORMED INTO PANDAS
    tbl = zip_precincts(t, list(dict)[0], dict[list(dict)[0]]).sort('precinct')
    for k in list(dict)[1:]:
        tbl = tbl.join('precinct', zip_precincts(t, k, dict[k]))
    return tbl

def save_file(t, name, idx=False):
    t.to_excel("/content/drive/My Drive/Notebooks/Bessin/Sheets/" + name + ".xlsx", index=idx, header=True)
    print('Saved as "' + name + '"')
    return None

breakdowns = {'sex': ['M', 'F', 'U'],
              'age': [(18, 25), (26, 49), (50, 65), (66, 80), (81, 100), (101, 300)],
              'party': ['D', 'R', 'U', 'G', 'L', 'P', 'O'],
              'race': ['Asian', 'Caucasian', 'Hispanic', 'Native American', 'Uncoded']}

In [ ]:
#import and setup data; enter file name, must be csv in Data folder
BVoters = table_setup('BVotersFull')
num_of_voters_full = 66873
linda_VANID = 7766030
kons_VANID = 15702787
tam_VANID = 7929336
schultz_VANID = 47146692
mano_VANID = 46376079
murphy_VANID = 7842797
machine_data_points = ['partisan', 'compassion', 'climatechange', 'tax', 'localvoter', 'progressivetax', 'incomeinequality', 'infrastructure', 'incomerank', 'minimumwage', 'midtermturnout', 'nonprespriturnout', 'trumpresistance']

In [ ]:
machine_data = BVoters[['VANID', 'lastname', 'firstname', 'sex', 'age', 'party', 'partisan', 'mailingaddress', 'city', 'state', 'zip', 'preferredemail', 'preferred Phone', 'precinct', 'authoritarian', 'compassion', 'climatechange', 'racialresentment', 'tax', 'localvoter', 'progressivetax', 'incomeinequality', 'infrastructure', 'incomerank', 'minimumwage', 'midtermturnout', 'nonprespriturnout', 'trumpresistance', 'trumpsupport']]
machine_data

In [ ]:
linda = machine_data[machine_data['VANID'] == linda_VANID].iloc[0]
kons = machine_data[machine_data['VANID'] == kons_VANID].iloc[0]
tam = machine_data[machine_data['VANID'] == tam_VANID].iloc[0]
schultz = machine_data[machine_data['VANID'] == schultz_VANID].iloc[0]
mano = machine_data[machine_data['VANID'] == mano_VANID].iloc[0]
murphy = machine_data[machine_data['VANID'] == murphy_VANID].iloc[0]

# linda_score functions
*36 lines*

In [ ]:
def linda_score(VANID):
    #requires existence of machine_data, which is derived from BVotersFull
    row = machine_data[machine_data['VANID'] == VANID].iloc[0]
    # row_series = machine_data[machine_data['VANID'] == VANID].iloc[0]
    if row['party'] in ['R', 'L', 'G', 'O', 'P']:
        if score_accuracy(VANID) == 0:
            return 0
        if row['partisan'] >= 50:
            return 0
        return 100 - linda_distance(VANID)
    if row['party'] == 'D':
        if score_accuracy(VANID) == 0:
            return 100
        if row['partisan'] >= 50:
            return 100 - (linda_distance(VANID) * .5)
        return 100 - linda_distance(VANID)
    if row['party'] == 'U':
        if score_accuracy(VANID) == 0:
            return 50
        if row['partisan'] >= 50:
            return 100 - (linda_distance(VANID) * 1.5)
        return 100 - linda_distance(VANID)

def linda_score_report(VANID):
    print('Linda Score: ' + str(linda_score(VANID)) + '\nLinda Score Accuracy: ' + str(score_accuracy(VANID)))

In [ ]:
def score_accuracy(VANID):
    #requires existence of machine_data, which is derived from BVotersFull
    row = machine_data[machine_data['VANID'] == VANID].iloc[0]
    row_spaces_filled = 0
    for i in machine_data_points:
        row_space_filled = row[i] == row[i]
        if row_space_filled:
            row_spaces_filled += 1
    return row_spaces_filled / len(machine_data_points)

In [ ]:
linda_score_report(vanid_finder('Weinberg', 'Diana'))

# Classify functions
*60 lines*

In [ ]:
def row_to_mdp_array(row):
    if type(row) == pd.core.series.Series:
        return make_array(row[machine_data_points])
    return make_array(row[machine_data_points].iloc[0])

def row_to_array(row):
    if type(row) == pd.core.series.Series:
        return make_array(row)
    return make_array(row.iloc[0])

def iloc_row_to_df(row):
    return pd.DataFrame(row, index=machine_data.columns).transpose()

def distance(array1, array2):
    row_distance = 0
    for i in range(len(array1)):
        row_distance += (array1.item(i) - array2.item(i)) ** 2
    return row_distance ** .5

def vanid_finder(last, first):
    return machine_data[machine_data['lastname'] == last][machine_data['firstname'] == first].iloc[0]['VANID']

def classify(VANID, train=iloc_row_to_df(linda).append(iloc_row_to_df(kons)).append(iloc_row_to_df(tam)).append(iloc_row_to_df(murphy))):
    row = machine_data[machine_data['VANID'] == VANID].iloc[0]
    test_row_features_array = row_to_mdp_array(row)
    distances = make_array()
    for i in range(4):
        train_row_features_array = row_to_mdp_array(train.iloc[i])
        distances = np.append(distances, distance(train_row_features_array, test_row_features_array))
    train['distance'] = distances
    nearest_neighbors = train.sort_values('distance')
    closest_label = nearest_neighbors.iloc[0]['firstname'] + ' ' + nearest_neighbors.iloc[0]['lastname']
    # print('Score Accuracy: ' + str(score_accuracy(VANID)))
    return closest_label

def distance_comps(VANID):
    row = machine_data[machine_data['VANID'] == VANID].iloc[0]
    test_row_features_array = row_to_mdp_array(row)
    distances = make_array()
    for i in range(4):
        train_row_features_array = row_to_mdp_array(train.iloc[i])
        distances = np.append(distances, distance(train_row_features_array, test_row_features_array))
    train['distance'] = distances
    return train[['firstname', 'lastname', 'distance']].sort_values('distance')

def linda_distance(VANID, train=iloc_row_to_df(linda).append(iloc_row_to_df(kons)).append(iloc_row_to_df(tam)).append(iloc_row_to_df(murphy))):
    row = machine_data[machine_data['VANID'] == VANID].iloc[0]
    test_row_features_array = row_to_mdp_array(row)
    distances = make_array()
    for i in range(4):
        train_row_features_array = row_to_mdp_array(train.iloc[i])
        distances = np.append(distances, distance(train_row_features_array, test_row_features_array))
    train['distance'] = distances
    # print('Score Accuracy: ' + str(score_accuracy(VANID)))
    return train.iloc[0]['distance']

In [ ]:
classify(vanid_finder('Kahn', 'Louie'))

In [ ]:
train = iloc_row_to_df(linda).append(iloc_row_to_df(kons)).append(iloc_row_to_df(tam)).append(iloc_row_to_df(murphy))
train[machine_data_points]

# Application to Machine Data

*The Ultimate Data Machine*

In [ ]:
def ultimate_data_machine(df=machine_data, columns=['VANID', 'lastname', 'firstname', 'sex', 'age', 'party', 'mailingaddress', 'city', 'state', 'zip', 'preferredemail', 'preferred Phone', 'precinct', 'linda_score', 'closest_dem_candidate', 'score_accuracy']):
    score_array = make_array()
    acc_array = make_array()
    exp_cand_array = make_array()
    for vanid in list(df['VANID']):
        score_array = np.append(score_array, linda_score(vanid))
        exp_cand_array = np.append(exp_cand_array, classify(vanid))
        acc_array = np.append(acc_array, score_accuracy(vanid))
    df['linda_score'] = score_array
    df['closest_dem_candidate'] = exp_cand_array
    df['score_accuracy'] = acc_array
    return df[columns]


In [ ]:
ultimate_data_machine(machine_data.take(range(50))) #IT WORKS!!